In [18]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import numpy as np


# Pfad zum Verzeichnis für die Dateien
INPUT_PATH_LOGS_DET = '../data/postprocessing/input_logs/det'
INPUT_PATH_LOGS_STO = '../data/postprocessing/input_logs/sto'

externer_pfad = "C:/Users/paul/Nextcloud/99_MATEX/MATEX/Inhalt/Abbildungen/plots"
os.makedirs(externer_pfad, exist_ok=True)


dates = ['20230316','20230409', '20230410', '20230501', '20230515', '20230619', '20230622', '20230709', '20230710']

# Output path for plots (create if doesn't exist)
OUTPUT_PATH = 'output_perfornance'
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)



In [19]:
def find_log_file(log_dir, date):
    pattern = re.compile(r'logfile_\d{8}_\d{6}_' + date + r'_day\.log')
    for filename in os.listdir(log_dir):
        if pattern.match(filename):
            return os.path.join(log_dir, filename)
    return None

In [20]:
def parse_gurobi_log(log_file_path):
    with open(log_file_path, 'r') as file:
        log_content = file.read()
    
    # Rechenzeit [s]
    time_match = re.search(r'in (\d+\.\d+) seconds', log_content)
    computation_time = float(time_match.group(1)) if time_match else None
    
    # Iterationen
    iter_match = re.search(r'(\d+) simplex iterations', log_content)
    iterations = int(iter_match.group(1)) if iter_match else None
    
    # Knoten
    nodes_match = re.search(r'Explored (\d+) nodes', log_content)
    nodes = int(nodes_match.group(1)) if nodes_match else None
    
    # Gap [%]
    gap_match = re.search(r'gap (\d+\.\d+)%', log_content)
    gap = float(gap_match.group(1)) if gap_match else None
    
    # Status
    if 'Optimal solution found' in log_content:
        status = 'Optimal'
    elif 'Infeasible' in log_content or 'Model is infeasible' in log_content:
        status = 'Unzulässig'
    else:
        status = 'Unbekannt'
    
    return {
        'Rechenzeit [s]': computation_time,
        'Iterationen': iterations,
        'Knoten': nodes,
        'Gap [%]': gap,
        'Status': status
    }


In [21]:
performance_records = []

for date in dates:
    # Deterministisches Modell
    det_log_file = find_log_file(INPUT_PATH_LOGS_DET, date)
    if det_log_file:
        det_performance = parse_gurobi_log(det_log_file)
        det_performance.update({'Datum': date, 'Modell': 'Deterministisch'})
        performance_records.append(det_performance)
    else:
        print(f"Keine deterministische Logdatei für Datum {date} gefunden.")
    
    # Stochastisches Modell
    sto_log_file = find_log_file(INPUT_PATH_LOGS_STO, date)
    if sto_log_file:
        sto_performance = parse_gurobi_log(sto_log_file)
        sto_performance.update({'Datum': date, 'Modell': 'Stochastisch'})
        performance_records.append(sto_performance)
    else:
        print(f"Keine stochastische Logdatei für Datum {date} gefunden.")

# Erstellen eines DataFrames mit den Performance-Daten
performance_df = pd.DataFrame(performance_records)

# Spaltenreihenfolge anpassen
performance_df = performance_df[['Datum', 'Modell', 'Rechenzeit [s]', 'Iterationen', 'Knoten', 'Gap [%]', 'Status']]

# Ergebnisse anzeigen
print("\nPerformance-Daten:")
print(performance_df)

# Tabelle als CSV speichern
performance_df.to_csv(os.path.join(OUTPUT_PATH, 'performance_data.csv'), index=False)



Performance-Daten:
       Datum           Modell  Rechenzeit [s]  Iterationen  Knoten  Gap [%]  \
0   20230316  Deterministisch            0.04          527       1   0.8095   
1   20230316     Stochastisch            0.28          402       1   0.1310   
2   20230409  Deterministisch            0.04          570       1   0.0000   
3   20230409     Stochastisch            0.22          522       1   0.1940   
4   20230410  Deterministisch            0.19         4642     879   0.9927   
5   20230410     Stochastisch            1.98       104508   19592   0.9967   
6   20230501  Deterministisch           29.26      1439337  221720   0.9959   
7   20230501     Stochastisch            0.30          538       1   0.7651   
8   20230515  Deterministisch            0.06          370       1   0.0004   
9   20230515     Stochastisch            0.24          415       1   0.0004   
10  20230619  Deterministisch            5.68       339144   84263   0.9998   
11  20230619     Stochastisch   

In [22]:
# Rechenzeit plotten
plt.figure(figsize=(10, 6))
sns.barplot(x='Datum', y='Rechenzeit [s]', hue='Modell', data=performance_df, palette='pastel')

plt.title('Rechenzeit der Modelle pro Tag')
plt.xlabel('Datum')
plt.ylabel('Rechenzeit [s]')
plt.legend(title='Modell')
plt.xticks(rotation=45)
plt.tight_layout()

# Plot speichern
plot_name = "rechenzeit_modelle_pro_tag.png"
extern = os.path.join(externer_pfad, plot_name)
plt.savefig(extern, dpi=300, bbox_inches='tight')
plt.close()


In [23]:
# Gap plotten
plt.figure(figsize=(10, 6))
sns.lineplot(x='Datum', y='Gap [%]', hue='Modell', data=performance_df, marker='o', palette='pastel')

plt.title('Gap der Modelle pro Tag')
plt.xlabel('Datum')
plt.ylabel('Gap [%]')
plt.legend(title='Modell')
plt.xticks(rotation=45)
plt.tight_layout()

# Plot speichern
plot_name = "gap_modelle_pro_tag.png"
extern = os.path.join(externer_pfad, plot_name)
plt.savefig(extern, dpi=300, bbox_inches='tight')
plt.close()


In [30]:
values = [-326.94, -364.97, -335.68, -296.03, -203.33, -155.04, -143.98, -156.93, -140.90]
# New set of values from the second image
third_values = [-331.49, -373.47, -315.20, -274.69, -170.51, -109.85, -109.54, -118.25, -126.16]

# Calculate the average for the third set of values
third_average = np.mean(third_values)
print("Third Average:", third_average)

# Calculate the difference between third_values and values
result = np.array(third_values) - np.array(values)
print("Result:", result)

result_average = np.mean(result)
print("Result Average:", result_average)

Third Average: -214.35111111111112
Result: [-4.55 -8.5  20.48 21.34 32.82 45.19 34.44 38.68 14.74]
Result Average: 21.626666666666665
